## 多输入多输出通道

当我们添加通道时，我们的输入和隐藏的表示都变成了三维张量。例如，每个RGB输入图像具有$3 \times h \times w$的形状。我们将这个大小为3的轴称为通道（channel）维度。在本节中，我们将更深入地研究具有多输入和多输出通道的卷积核。

In [1]:
import torch
import sys
sys.path.append('..')
import d2l

### 1.多输入通道，单输出通道

- 每个输入通道都有一个卷积核，结果是所有通道卷积结果的和；
- 卷积核应相应地扩展为三阶：(通道维, 高, 宽)；
- 相应的卷积操作：

$$
[H]_{i,j} = u + \sum_c\sum_{a=-\Delta}^{\Delta}\sum_{b=-\Delta}^{\Delta}[V]_{c,a,b}[X]_{i+a,j+b}
$$

In [2]:
def corr2d_multi_in(X, K):
    '''多通道输入的2d互相关运算，此时卷积核K为3d'''
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

![](https://zh-v2.d2l.ai/_images/conv-multi-in.svg)

In [3]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

### 2.多输出通道

- 特定输出通道识别特定模式；
- 每个输出通道都有一个3d卷积核，每个3d卷积核生成一个输出通道；
- 卷积核相应地扩展为四阶：(输出通道维$c_o$，输入通道维$c_i$，高，宽)；
- 相应的卷积操作：

$$
[H]_{d,i,j} = u + \sum_c\sum_{a=-\Delta}^{\Delta}\sum_{b=-\Delta}^{\Delta}[V]_{d,c,a,b}[X]_{i+a,j+b}
$$
- 最终计算复杂度：$O(c_ic_ok_hk_wm_hm_w)$，单位为浮点计算数FLOPS，$m_h$和$m_w$分别为输出的高度和宽度。


In [5]:
def corr2d_multi_in_out(X, K):
    '''多输入多输出通道的2d互相关运算，此时卷积核K为4d'''
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)   # list->第0维压缩成张量

K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [6]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 3. $1 \times 1$的2d卷积

- $k_h = k_w = 1$的卷积是一个受欢迎的选择。它不识别空间模式，只融合通道；
- 相当于输入形状为$(c_i, ~ n_h*n_w)$，权重形状为$(c_o, ~ c_i)$的全连接层

In [8]:
def corr2d_multi_in_out_1x1(X, K):
    '''多输入多输出通道的1*1卷积，使用全连接方式实现'''
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [9]:
# 验证等价
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

### 4.总结

- 输出通道数是卷积层的超参数；
- 每个输入通道有独立的二维卷积核，所有输入通道的卷积结果相加得到一个输出通道的结果；
- 每个输出通道有独立的三维卷积核。